# Performance comparison between *iarray* and *numexpr*

In [ ]:
import iarray as ia
import numpy as np
import numexpr as ne

ModuleNotFoundError: No module named 'iarray'

In [2]:
# In order to compare pears with pears we set the number of threads in numexpr to 1
_ = ne.set_num_threads(1)

In [3]:
ia.init()

cfg = ia.config_new(eval_flags="iterblock", blocksize=16*1024, compression_level=0, compression_codec=1)
ctx = ia.context_new(cfg)

In [4]:
# Vectors properties
shape = [10 * 200 * 1000]
pshape = [200 * 1000]
size = int(np.prod(shape))

In [5]:
# Create iarray containers
a = ia.linspace(ctx, size, 0, 10, pshape=pshape)
out = ia.container_new(ctx, shape, pshape)

# Create iarray expression
expr = ia.expr_new(ctx)

# Bind iarray containers
ia.expr_bind(expr, b'x', a)

# Compile a+b expression
ia.expr_compile(expr, b'(x - 1.35) * (x - 4.45) * (x - 8.5)')

# Eval expression
%timeit ia.expr_eval(expr, out)

22.8 ms ± 1.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
# Create a numpy array for numexpr
x = ia.iarray2numpy(ctx, a)
out2 = np.empty(x.shape, dtype=x.dtype)

# Eval numexpr expression
%timeit ne.evaluate("(x - 1.35) * (x - 4.45) * (x - 8.5)", out=out2)

7.73 ms ± 688 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
np.testing.assert_almost_equal(ia.iarray2numpy(ctx, out), out2)

In [8]:
# Free data
ia.expr_free(ctx, expr)
ia.container_free(ctx, a)
ia.container_free(ctx, out)
ia.context_free(ctx)

# Destroy iarray
ia.destroy()